In [ ]:
# Programa em Python para analisar dados de uma planilha Excel, remover outliers, gerar um gráfico e salvá-lo para impressão.
# Agora convertido para um formato de Jupyter Notebook para facilitar a interatividade.

# Importar bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os

# Parte 1: Carregar e filtrar dados do Excel

# Definindo caminho relativo baseado na estrutura do projeto
caminho_arquivo = os.path.join(os.getcwd(), "../data/raw/dadosprocessados/dados_processados.xlsx")
aba = "Dados Processados"

# Carregar os dados
if not os.path.isfile(caminho_arquivo):
    raise FileNotFoundError(f"Arquivo não encontrado: {caminho_arquivo}")

try:
    print(f"Carregando dados do arquivo: {caminho_arquivo}")
    xls = pd.ExcelFile(caminho_arquivo, engine='openpyxl')
    print(f"Abas disponíveis no arquivo: {xls.sheet_names}")
    df = pd.read_excel(xls, sheet_name=aba)
except PermissionError as e:
    raise PermissionError(f"Permissão negada ao tentar acessar o arquivo: {caminho_arquivo}. Verifique as permissões do arquivo.")
except Exception as e:
    raise Exception(f"Erro inesperado ao carregar o arquivo Excel: {str(e)}")

# Filtrar os dados removendo outliers
limite_outlier = 100
df_filtrado = df[df["MÉDIA PONDERADA (%)"] <= limite_outlier]

# Parte 2: Calcular a linha de tendência

# Função para calcular a linha de tendência usando regressão linear
coluna_x = "COD. BATIDA"
coluna_y = "MÉDIA PONDERADA (%)"

z = np.polyfit(df_filtrado[coluna_x], np.log10(df_filtrado[coluna_y]), 1)
linha_tendencia = np.poly1d(z)

# Parte 3: Criar e visualizar o gráfico
titulo = "Gráfico de Linha: Média Ponderada (%) vs COD. BATIDA (Escala Log10) - Sem Valores > 10²"
periodo_inicio = df_filtrado["COD. BATIDA"].iloc[0]
periodo_fim = df_filtrado["COD. BATIDA"].iloc[-1]
periodo_texto = f"Análise CÓD. BATIDA de {periodo_inicio} a {periodo_fim}"

plt.figure(figsize=(12, 8), dpi=300)

# Plotar a linha dos dados filtrados
plt.plot(
    df_filtrado[coluna_x],
    df_filtrado[coluna_y],
    label="Média Ponderada (%)",
    color="dimgray",
    linewidth=1.5,
)

# Configurar o eixo Y para escala logarítmica
plt.yscale("log")

# Plotar a linha de tendência
plt.plot(
    df_filtrado[coluna_x],
    10 ** linha_tendencia(df_filtrado[coluna_x]),
    color="red",
    linestyle="-",
    linewidth=3.0,
    label="Linha de Tendência",
)

# Configurar elementos visuais do gráfico
plt.title(titulo, fontsize=16, fontweight="bold")
plt.xlabel(coluna_x, fontsize=14, fontweight="bold")
plt.ylabel(f"{coluna_y} (Escala Log10)", fontsize=14, fontweight="bold")
plt.legend(fontsize=12)
plt.grid(True, linestyle="--", linewidth=0.7, color="gray")
plt.gca().set_facecolor("white")

# Inserir o texto do período na margem inferior
plt.figtext(0.5, 0.01, periodo_texto, ha="center", fontsize=12, fontweight="bold")

# Parte 4: Salvar e mostrar o gráfico
data_producao = datetime.now().strftime("%Y-%m-%d")
caminho_saida = os.path.join(os.getcwd(), f"../data/processed/graphics/trendline_{data_producao}.png")

# Salvar o gráfico antes de exibir
try:
    plt.savefig(caminho_saida, format="png", bbox_inches="tight", pad_inches=0.2)
    print(f"Gráfico salvo em: {caminho_saida}")
except PermissionError:
    print(f"Permissão negada ao tentar salvar o arquivo: {caminho_saida}. Verifique as permissões do diretório.")
except Exception as e:
    print(f"Erro inesperado ao salvar o gráfico: {str(e)}")

# Mostrar o gráfico
plt.show()

# Fechar o gráfico para liberar recursos
plt.close()
